In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/cumulative.csv')
data.head()

In [ ]:
archive_info = ['kepler_name', 'kepid', 'kepoi_name', 'koi_disposition', 'koi_pdisposition', 'rowid']
data = data.drop(archive_info, axis=1)

In [ ]:
project_disposition = ['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec']
data = data.drop(project_disposition, axis=1)

In [ ]:
threshold_crossing_events = ['koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname']
data = data.drop(threshold_crossing_events, axis=1)

In [ ]:
kic_parameters = ['ra', 'dec', 'koi_kepmag']
data = data.drop(kic_parameters, axis=1)

In [ ]:
transit_properties = ['koi_period_err1', 'koi_period_err2', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration_err1', 
                      'koi_duration_err2', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol_err1', 
                      'koi_insol_err2', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad_err1', 'koi_srad_err2']
data = data.drop(transit_properties, axis=1)

In [ ]:
pd.set_option('display.max_columns', 50)

data.dropna(inplace=True)
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
features = ['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_srad']
data_features = data[features]

data_features.head()

In [ ]:
data_target = data['koi_score']

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import numpy as np

In [ ]:
class ExoplanetDataset(torch.utils.data.Dataset):

    def __init__(self, train=True):
        x_train, x_test, y_train, y_test = train_test_split(torch.from_numpy(data_features.values).float(), 
                                                            torch.from_numpy(data_target.values).float(), 
                                                            test_size=0.2, 
                                                            random_state=0)
        
        if train:
            self.x_data, self.y_data = x_train, y_train
        else:
            self.x_data, self.y_data = x_test, y_test

    def __len__(self):
        return self.y_data.shape[0]
    
    def __getitem__(self, i):
        return self.x_data[i], self.y_data[i]

In [ ]:
train_loader = torch.utils.data.DataLoader(ExoplanetDataset(train=True), batch_size=12)
test_loader = torch.utils.data.DataLoader(ExoplanetDataset(train=False), batch_size=12)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
input_size = 11
output_size = 1
hid1_size = 20
hid2_size = 10

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hid1_size)
    self.fc2 = nn.Linear(hid1_size, hid2_size)
    self.fc3 = nn.Linear(hid2_size, output_size)
  
  def forward(self, x):
    x = torch.sigmoid(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    x = self.fc3(x)

    return torch.sigmoid(x)

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss(reduction='mean')

In [ ]:
epoch_data = []
epochs = 1000

In [67]:
for epoch in range(1, epochs + 1):
    valid_loss = 0
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        output_train = model(x)
        loss = loss_fn(output_train.squeeze(1), y)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            output_test = model(x)
            loss = loss_fn(output_test.squeeze(1), y)
            valid_loss += loss.item()
            valid_loss /= len(test_loader)

    if epoch % 100 == 0:
        print(f"Training loss: {loss.item()} \| Valid loss: {valid_loss}")
    

Training loss: 0.18527726829051971 \| Valid loss: 0.001392566091486623
Training loss: 0.18398189544677734 \| Valid loss: 0.0013870735553432945
Training loss: 0.18216216564178467 \| Valid loss: 0.0013688187397580512
Training loss: 0.18602077662944794 \| Valid loss: 0.0013977914381117173
Training loss: 0.18485140800476074 \| Valid loss: 0.0013892560240616322
Training loss: 0.1849900484085083 \| Valid loss: 0.0013903991405793644
Training loss: 0.18614010512828827 \| Valid loss: 0.0013986333558347866
Training loss: 0.18308742344379425 \| Valid loss: 0.0013757235661478397
Training loss: 0.18315689265727997 \| Valid loss: 0.001376241912670444
Training loss: 0.1804952621459961 \| Valid loss: 0.0013565849588303342
